# Overfitting et Regularisation

> Ce notebook est un exemple pratique clef en main pour comprendre visuellement l'overfitting et maitriser les techniques de regularisation.

Ce notebook demontre visuellement l'overfitting et les techniques de regularisation.

## Objectifs

- Comprendre ce qu'est l'**overfitting** (surapprentissage) et le detecter
- Appliquer le **Dropout** comme technique de regularisation
- Utiliser le **Weight Decay** (regularisation L2)
- Implementer l'**Early Stopping**
- Decouvrir la **Data Augmentation**
- Comparer toutes les approches

### Pre-requis

```bash
uv add torch torchvision matplotlib numpy scikit-learn
```

In [ ]:
# === Imports ===
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from copy import deepcopy

# Configuration
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device utilise : {device}")

# Graine aleatoire pour la reproductibilite
torch.manual_seed(42)
np.random.seed(42)

---
## Section 1 - Generer un Dataset Synthetique

Nous utilisons `make_moons` de scikit-learn pour creer un dataset de classification 2D en forme de croissants de lune. Ce dataset est volontairement **petit** (300 points) et **bruite** pour provoquer facilement l'overfitting.

> **Pourquoi un petit dataset ?** L'overfitting se produit lorsque le modele a trop de parametres par rapport a la quantite de donnees. Avec un petit dataset, c'est plus facile a observer.

In [ ]:
# === Generation du dataset ===

NB_ECHANTILLONS = 300
BRUIT = 0.25  # Quantite de bruit (rend la classification plus difficile)

# Creer les donnees en forme de croissants de lune
X, y = make_moons(n_samples=NB_ECHANTILLONS, noise=BRUIT, random_state=42)

# Separer en ensembles d'entrainement (60%), validation (20%), test (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Taille de l'ensemble d'entrainement : {len(X_train)}")
print(f"Taille de l'ensemble de validation  : {len(X_val)}")
print(f"Taille de l'ensemble de test        : {len(X_test)}")

# Convertir en tensors PyTorch
X_train_t = torch.FloatTensor(X_train).to(device)
y_train_t = torch.LongTensor(y_train).to(device)
X_val_t = torch.FloatTensor(X_val).to(device)
y_val_t = torch.LongTensor(y_val).to(device)
X_test_t = torch.FloatTensor(X_test).to(device)
y_test_t = torch.LongTensor(y_test).to(device)

# DataLoaders
train_dataset = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# === Visualisation du dataset ===

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, X_set, y_set, titre in [
    (axes[0], X_train, y_train, f'Entrainement (n={len(X_train)})'),
    (axes[1], X_val, y_val, f'Validation (n={len(X_val)})'),
    (axes[2], X_test, y_test, f'Test (n={len(X_test)})'),
]:
    scatter = ax.scatter(X_set[:, 0], X_set[:, 1], c=y_set, cmap='RdYlBu',
                         edgecolors='black', s=50, alpha=0.8)
    ax.set_title(titre, fontsize=14, fontweight='bold')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.grid(True, alpha=0.3)

plt.suptitle('Dataset "Make Moons" - Classification binaire 2D', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

---
## Section 2 - Un Modele qui Overfit

Nous allons maintenant creer un reseau **beaucoup trop gros** pour nos 180 points d'entrainement. Avec des centaines de milliers de parametres pour si peu de donnees, le modele va **memoriser** les donnees d'entrainement au lieu d'apprendre le pattern general.

> **Signe d'overfitting** : La perte d'entrainement diminue, mais la perte de validation **augmente** apres un certain point.

In [ ]:
# === Fonctions utilitaires ===

def entrainer_modele(modele, optimiseur, nb_epochs=200, verbose=True):
    """Entraine un modele et retourne l'historique des pertes."""
    critere = nn.CrossEntropyLoss()
    historique = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(nb_epochs):
        # --- Entrainement ---
        modele.train()
        total_loss, correct, total = 0, 0, 0

        for X_batch, y_batch in train_loader:
            predictions = modele(X_batch)
            loss = critere(predictions, y_batch)
            optimiseur.zero_grad()
            loss.backward()
            optimiseur.step()

            total_loss += loss.item() * X_batch.size(0)
            _, preds = torch.max(predictions, 1)
            total += y_batch.size(0)
            correct += (preds == y_batch).sum().item()

        train_loss = total_loss / total
        train_acc = 100 * correct / total

        # --- Validation ---
        modele.eval()
        with torch.no_grad():
            val_preds = modele(X_val_t)
            val_loss = critere(val_preds, y_val_t).item()
            _, val_predicted = torch.max(val_preds, 1)
            val_acc = 100 * (val_predicted == y_val_t).sum().item() / len(y_val_t)

        historique['train_loss'].append(train_loss)
        historique['val_loss'].append(val_loss)
        historique['train_acc'].append(train_acc)
        historique['val_acc'].append(val_acc)

        if verbose and (epoch + 1) % 50 == 0:
            print(f"  Epoch {epoch+1:>3}/{nb_epochs} | "
                  f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
                  f"Train Acc: {train_acc:.1f}% | Val Acc: {val_acc:.1f}%")

    return historique


def tracer_courbes(historique, titre=""):
    """Trace les courbes de perte et precision (train vs validation)."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    epochs = range(1, len(historique['train_loss']) + 1)

    # Courbe de perte
    ax1.plot(epochs, historique['train_loss'], 'b-', label='Train', linewidth=2, alpha=0.8)
    ax1.plot(epochs, historique['val_loss'], 'r-', label='Validation', linewidth=2, alpha=0.8)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Perte (Loss)')
    ax1.set_title('Perte')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Courbe de precision
    ax2.plot(epochs, historique['train_acc'], 'b-', label='Train', linewidth=2, alpha=0.8)
    ax2.plot(epochs, historique['val_acc'], 'r-', label='Validation', linewidth=2, alpha=0.8)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Precision (%)')
    ax2.set_title('Precision')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.suptitle(titre, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


def tracer_frontiere_decision(modele, X, y, titre=""):
    """Trace la frontiere de decision du modele sur les donnees."""
    modele.eval()

    # Creer une grille de points
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))

    # Predire sur toute la grille
    grille = torch.FloatTensor(np.c_[xx.ravel(), yy.ravel()]).to(device)
    with torch.no_grad():
        Z = modele(grille)
        Z = torch.softmax(Z, dim=1)[:, 1]  # Probabilite de la classe 1
    Z = Z.cpu().numpy().reshape(xx.shape)

    # Tracer
    fig, ax = plt.subplots(figsize=(10, 7))
    contour = ax.contourf(xx, yy, Z, levels=50, cmap='RdYlBu', alpha=0.6)
    ax.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)  # Frontiere
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap='RdYlBu', edgecolors='black', s=60)
    plt.colorbar(contour, ax=ax, label='Probabilite classe 1')
    ax.set_title(titre, fontsize=14, fontweight='bold')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    plt.tight_layout()
    plt.show()

In [ ]:
# === Modele TROP GRAND (provoque l'overfitting) ===

class GrosModele(nn.Module):
    """Un reseau beaucoup trop gros pour 180 points d'entrainement."""

    def __init__(self):
        super(GrosModele, self).__init__()
        self.couches = nn.Sequential(
            nn.Linear(2, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
        )

    def forward(self, x):
        return self.couches(x)


# Entrainer le gros modele
modele_overfit = GrosModele().to(device)
nb_params = sum(p.numel() for p in modele_overfit.parameters())
print(f"Nombre de parametres : {nb_params:,} (pour seulement {len(X_train)} points !)")
print(f"Ratio parametres/donnees : {nb_params/len(X_train):.0f}:1 (devrait etre < 10:1)")
print()

optimiseur_overfit = optim.Adam(modele_overfit.parameters(), lr=0.001)

print("Entrainement du modele (sans regularisation) :")
historique_overfit = entrainer_modele(modele_overfit, optimiseur_overfit, nb_epochs=200)

# Visualiser les courbes
tracer_courbes(historique_overfit, 'SANS regularisation - Overfitting visible')

# Visualiser la frontiere de decision
tracer_frontiere_decision(modele_overfit, X_train, y_train,
                          'Frontiere de decision - Modele surappris (overfitted)')

---
## Section 3 - Regularisation par Dropout

Le **Dropout** est une technique de regularisation qui consiste a **desactiver aleatoirement** un pourcentage de neurones a chaque etape d'entrainement.

**Comment ca fonctionne :**
- Pendant l'entrainement : chaque neurone a une probabilite `p` d'etre mis a zero
- Pendant l'evaluation : tous les neurones sont actifs, mais les poids sont multiplies par `(1-p)` pour compenser

**Pourquoi ca marche :** Le reseau ne peut pas compter sur un seul chemin neural. Il doit apprendre des representations **redondantes** et **robustes**.

In [ ]:
# === Modele avec Dropout ===

class GrosModeleDropout(nn.Module):
    """Meme architecture que GrosModele, mais avec Dropout."""

    def __init__(self, dropout_rate=0.4):
        super(GrosModeleDropout, self).__init__()
        self.couches = nn.Sequential(
            nn.Linear(2, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),    # 40% des neurones desactives
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 2),
        )

    def forward(self, x):
        return self.couches(x)


modele_dropout = GrosModeleDropout(dropout_rate=0.4).to(device)
optimiseur_dropout = optim.Adam(modele_dropout.parameters(), lr=0.001)

print("Entrainement du modele avec Dropout (p=0.4) :")
historique_dropout = entrainer_modele(modele_dropout, optimiseur_dropout, nb_epochs=200)

# Visualiser
tracer_courbes(historique_dropout, 'AVEC Dropout (p=0.4) - Meilleure generalisation')
tracer_frontiere_decision(modele_dropout, X_train, y_train,
                          'Frontiere de decision - Avec Dropout')

---
## Section 4 - Regularisation par Weight Decay

Le **Weight Decay** (ou regularisation L2) ajoute une penalite sur la taille des poids dans la fonction de perte :

$$\text{Loss}_{\text{total}} = \text{Loss}_{\text{original}} + \lambda \sum_{i} w_i^2$$

Cela force les poids a rester **petits**, ce qui produit des modeles plus simples et mieux generalises.

En PyTorch, il suffit d'ajouter le parametre `weight_decay` a l'optimiseur.

In [ ]:
# === Modele avec Weight Decay ===

modele_wd = GrosModele().to(device)  # Meme architecture sans Dropout

# Le weight_decay est le parametre lambda de la regularisation L2
WEIGHT_DECAY = 0.01  # Valeur typique : entre 0.0001 et 0.1

optimiseur_wd = optim.Adam(modele_wd.parameters(), lr=0.001, weight_decay=WEIGHT_DECAY)

print(f"Entrainement du modele avec Weight Decay (lambda={WEIGHT_DECAY}) :")
historique_wd = entrainer_modele(modele_wd, optimiseur_wd, nb_epochs=200)

# Visualiser
tracer_courbes(historique_wd, f'AVEC Weight Decay (lambda={WEIGHT_DECAY})')
tracer_frontiere_decision(modele_wd, X_train, y_train,
                          'Frontiere de decision - Avec Weight Decay')

# Comparer la norme des poids
norme_overfit = sum(p.data.norm().item()**2 for p in modele_overfit.parameters())**0.5
norme_wd = sum(p.data.norm().item()**2 for p in modele_wd.parameters())**0.5
print(f"\nNorme des poids (sans regularisation) : {norme_overfit:.2f}")
print(f"Norme des poids (avec Weight Decay)   : {norme_wd:.2f}")
print(f"Reduction : {(1 - norme_wd/norme_overfit)*100:.0f}%")

---
## Section 5 - Early Stopping

L'**Early Stopping** est la technique la plus simple et souvent la plus efficace pour eviter l'overfitting.

**Principe :** On surveille la perte de validation pendant l'entrainement. Si elle ne s'ameliore plus pendant un certain nombre d'epochs (**patience**), on arrete et on revient au meilleur modele.

```
Perte de validation :
  |\        /
  | \      /  <-- Overfitting commence ici
  |  \    /
  |   \__/    <-- On s'arrete au minimum
  +---------> Epochs
```

In [ ]:
# === Entrainement avec Early Stopping ===

def entrainer_avec_early_stopping(modele, optimiseur, patience=15, nb_epochs_max=300):
    """Entraine un modele avec Early Stopping.

    Args:
        patience: nombre d'epochs sans amelioration avant d'arreter
        nb_epochs_max: nombre maximal d'epochs
    """
    critere = nn.CrossEntropyLoss()
    historique = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    meilleure_val_loss = float('inf')
    meilleur_modele = None
    compteur_patience = 0
    epoch_arret = nb_epochs_max

    for epoch in range(nb_epochs_max):
        # --- Entrainement ---
        modele.train()
        total_loss, correct, total = 0, 0, 0

        for X_batch, y_batch in train_loader:
            predictions = modele(X_batch)
            loss = critere(predictions, y_batch)
            optimiseur.zero_grad()
            loss.backward()
            optimiseur.step()

            total_loss += loss.item() * X_batch.size(0)
            _, preds = torch.max(predictions, 1)
            total += y_batch.size(0)
            correct += (preds == y_batch).sum().item()

        train_loss = total_loss / total
        train_acc = 100 * correct / total

        # --- Validation ---
        modele.eval()
        with torch.no_grad():
            val_preds = modele(X_val_t)
            val_loss = critere(val_preds, y_val_t).item()
            _, val_predicted = torch.max(val_preds, 1)
            val_acc = 100 * (val_predicted == y_val_t).sum().item() / len(y_val_t)

        historique['train_loss'].append(train_loss)
        historique['val_loss'].append(val_loss)
        historique['train_acc'].append(train_acc)
        historique['val_acc'].append(val_acc)

        # --- Logique Early Stopping ---
        if val_loss < meilleure_val_loss:
            meilleure_val_loss = val_loss
            meilleur_modele = deepcopy(modele.state_dict())  # Sauvegarder le meilleur modele
            compteur_patience = 0
        else:
            compteur_patience += 1

        if compteur_patience >= patience:
            epoch_arret = epoch + 1
            print(f"  Early Stopping declenchee a l'epoch {epoch_arret} !")
            print(f"  Meilleure val loss : {meilleure_val_loss:.4f} (a l'epoch {epoch_arret - patience})")
            break

        if (epoch + 1) % 50 == 0:
            print(f"  Epoch {epoch+1:>3}/{nb_epochs_max} | "
                  f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
                  f"Patience: {compteur_patience}/{patience}")

    # Restaurer le meilleur modele
    if meilleur_modele is not None:
        modele.load_state_dict(meilleur_modele)

    return historique, epoch_arret


# Entrainer avec Early Stopping
modele_es = GrosModele().to(device)
optimiseur_es = optim.Adam(modele_es.parameters(), lr=0.001)

PATIENCE = 15
print(f"Entrainement avec Early Stopping (patience={PATIENCE}) :")
historique_es, epoch_arret = entrainer_avec_early_stopping(
    modele_es, optimiseur_es, patience=PATIENCE, nb_epochs_max=300
)

# Visualiser avec la ligne d'arret
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

epochs = range(1, len(historique_es['train_loss']) + 1)

ax1.plot(epochs, historique_es['train_loss'], 'b-', label='Train', linewidth=2)
ax1.plot(epochs, historique_es['val_loss'], 'r-', label='Validation', linewidth=2)
meilleure_epoch = epoch_arret - PATIENCE
if meilleure_epoch > 0:
    ax1.axvline(x=meilleure_epoch, color='green', linestyle='--', linewidth=2,
                label=f'Meilleur modele (epoch {meilleure_epoch})')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Perte')
ax1.set_title('Perte avec Early Stopping')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(epochs, historique_es['train_acc'], 'b-', label='Train', linewidth=2)
ax2.plot(epochs, historique_es['val_acc'], 'r-', label='Validation', linewidth=2)
if meilleure_epoch > 0:
    ax2.axvline(x=meilleure_epoch, color='green', linestyle='--', linewidth=2,
                label=f'Meilleur modele (epoch {meilleure_epoch})')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Precision (%)')
ax2.set_title('Precision avec Early Stopping')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle(f'Early Stopping (patience={PATIENCE}, arret epoch {epoch_arret})',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

tracer_frontiere_decision(modele_es, X_train, y_train,
                          f'Frontiere de decision - Early Stopping (epoch {meilleure_epoch})')

---
## Section 6 - Data Augmentation (concept)

La **Data Augmentation** consiste a creer de nouvelles donnees d'entrainement en appliquant des **transformations** aux donnees existantes. C'est particulierement utile pour les images.

Transformations courantes :
- **Rotation** aleatoire
- **Flip** horizontal/vertical
- **Crop** (recadrage) aleatoire
- **Changement de luminosite/contraste**
- **Ajout de bruit**

> **Principe :** Plus on a de donnees (meme synthetiques), moins le modele risque d'overfitter.

In [ ]:
# === Demonstration de la Data Augmentation sur des images ===

# Charger quelques images MNIST pour la demonstration
mnist_demo = torchvision.datasets.MNIST(
    root='./data', train=True, download=True,
    transform=transforms.ToTensor()
)

# Definir differentes transformations
augmentations = {
    'Original': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
    ]),
    'Rotation\n(+/- 30 deg)': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
    ]),
    'Translation\naleatoire': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),
        transforms.ToTensor(),
    ]),
    'Mise a\nl\'echelle': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=0, scale=(0.7, 1.3)),
        transforms.ToTensor(),
    ]),
    'Deformation\nperspective': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomPerspective(distortion_scale=0.4, p=1.0),
        transforms.ToTensor(),
    ]),
    'Effacement\naleatoire': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.RandomErasing(p=1.0, scale=(0.05, 0.2)),
    ]),
}

# Afficher les augmentations pour 3 images differentes
nb_images = 3
nb_transforms = len(augmentations)

fig, axes = plt.subplots(nb_images, nb_transforms, figsize=(16, 8))
fig.suptitle('Exemples de Data Augmentation sur MNIST', fontsize=16, fontweight='bold')

for i in range(nb_images):
    image_originale = mnist_demo[i][0]  # Tensor de l'image

    for j, (nom, transform) in enumerate(augmentations.items()):
        img_augmentee = transform(image_originale)
        axes[i, j].imshow(img_augmentee.squeeze().numpy(), cmap='gray')
        if i == 0:
            axes[i, j].set_title(nom, fontsize=10, fontweight='bold')
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()

print("\nLa Data Augmentation multiplie artificiellement la taille du dataset.")
print("Chaque epoch, le modele voit des versions legerement differentes des images,")
print("ce qui l'empeche de memoriser les exemples exacts.")

---
## Section 7 - Comparaison Finale

Comparons toutes les approches de regularisation sur notre dataset synthetique.

In [ ]:
# === Evaluation finale de tous les modeles sur le jeu de TEST ===

def evaluer_sur_test(modele, nom):
    """Evalue un modele sur le jeu de test et retourne les resultats."""
    modele.eval()
    critere = nn.CrossEntropyLoss()

    with torch.no_grad():
        preds = modele(X_test_t)
        loss = critere(preds, y_test_t).item()
        _, predicted = torch.max(preds, 1)
        acc = 100 * (predicted == y_test_t).sum().item() / len(y_test_t)

    return {'nom': nom, 'test_loss': loss, 'test_acc': acc}


# Evaluer chaque modele
resultats = [
    evaluer_sur_test(modele_overfit, 'Sans regularisation'),
    evaluer_sur_test(modele_dropout, 'Dropout (p=0.4)'),
    evaluer_sur_test(modele_wd, f'Weight Decay ({WEIGHT_DECAY})'),
    evaluer_sur_test(modele_es, f'Early Stopping (p={PATIENCE})'),
]

# Tableau recapitulatif
print("=" * 65)
print(f"{'Methode':<30} | {'Test Loss':>12} | {'Test Accuracy':>14}")
print("=" * 65)
for r in resultats:
    print(f"{r['nom']:<30} | {r['test_loss']:>12.4f} | {r['test_acc']:>13.1f}%")
print("=" * 65)

# === Graphique de comparaison ===
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

noms = [r['nom'] for r in resultats]
couleurs = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12']

# Precision
accs = [r['test_acc'] for r in resultats]
bars1 = ax1.bar(range(len(noms)), accs, color=couleurs, edgecolor='black', width=0.6)
ax1.set_ylabel('Precision sur le test (%)')
ax1.set_title('Precision sur le jeu de test', fontweight='bold')
ax1.set_xticks(range(len(noms)))
ax1.set_xticklabels(noms, rotation=20, ha='right', fontsize=10)
ax1.grid(True, alpha=0.3, axis='y')
for bar, acc in zip(bars1, accs):
    ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.3,
             f'{acc:.1f}%', ha='center', va='bottom', fontweight='bold')

# Perte
losses = [r['test_loss'] for r in resultats]
bars2 = ax2.bar(range(len(noms)), losses, color=couleurs, edgecolor='black', width=0.6)
ax2.set_ylabel('Perte sur le test')
ax2.set_title('Perte sur le jeu de test', fontweight='bold')
ax2.set_xticks(range(len(noms)))
ax2.set_xticklabels(noms, rotation=20, ha='right', fontsize=10)
ax2.grid(True, alpha=0.3, axis='y')
for bar, loss in zip(bars2, losses):
    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.005,
             f'{loss:.3f}', ha='center', va='bottom', fontweight='bold')

plt.suptitle('Comparaison des techniques de regularisation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# === Comparaison des courbes de validation ===
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

historiques = [
    (historique_overfit, 'Sans regularisation', '#e74c3c'),
    (historique_dropout, 'Dropout', '#3498db'),
    (historique_wd, 'Weight Decay', '#2ecc71'),
    (historique_es, 'Early Stopping', '#f39c12'),
]

for hist, nom, couleur in historiques:
    epochs = range(1, len(hist['val_loss']) + 1)
    ax1.plot(epochs, hist['val_loss'], label=nom, linewidth=2, color=couleur, alpha=0.8)
    ax2.plot(epochs, hist['val_acc'], label=nom, linewidth=2, color=couleur, alpha=0.8)

ax1.set_xlabel('Epoch')
ax1.set_ylabel('Perte de validation')
ax1.set_title('Evolution de la perte de validation')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Epoch')
ax2.set_ylabel('Precision de validation (%)')
ax2.set_title('Evolution de la precision de validation')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('Courbes de validation - Toutes les approches', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# === Comparaison des frontieres de decision ===
fig, axes = plt.subplots(1, 4, figsize=(24, 5))
modeles = [
    (modele_overfit, 'Sans regularisation'),
    (modele_dropout, 'Dropout'),
    (modele_wd, 'Weight Decay'),
    (modele_es, 'Early Stopping'),
]

x_min, x_max = X_train[:, 0].min() - 0.5, X_train[:, 0].max() + 0.5
y_min, y_max = X_train[:, 1].min() - 0.5, X_train[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))
grille = torch.FloatTensor(np.c_[xx.ravel(), yy.ravel()]).to(device)

for ax, (mod, nom) in zip(axes, modeles):
    mod.eval()
    with torch.no_grad():
        Z = mod(grille)
        Z = torch.softmax(Z, dim=1)[:, 1].cpu().numpy().reshape(xx.shape)

    ax.contourf(xx, yy, Z, levels=50, cmap='RdYlBu', alpha=0.6)
    ax.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap='RdYlBu',
               edgecolors='black', s=40)
    ax.set_title(nom, fontsize=12, fontweight='bold')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')

plt.suptitle('Frontieres de decision comparees (sur donnees de TEST)',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## Conclusion

### Resume des techniques de regularisation

| Technique | Principe | Quand l'utiliser |
|---|---|---|
| **Dropout** | Desactive aleatoirement des neurones | Reseaux denses (MLP), facile a ajouter |
| **Weight Decay (L2)** | Penalise les grands poids | Toujours, en premiere intention |
| **Early Stopping** | Arrete l'entrainement au bon moment | Toujours, indispensable en pratique |
| **Data Augmentation** | Augmente artificiellement le dataset | Vision par ordinateur surtout |
| **Batch Normalization** | Normalise les activations | Reseaux profonds (> 5 couches) |

---

### Conseil de pro : Comment detecter l'overfitting ?

> **Toujours tracer les courbes train vs validation !** C'est le moyen le plus fiable.
> - Si les courbes **divergent** (train descend, val remonte) = overfitting
> - Si les courbes **restent proches** = bonne generalisation
> - Si les deux courbes sont **hautes** = underfitting (modele trop simple)

### Conseil de pro : Quelle technique choisir ?

> En pratique, on **combine** souvent plusieurs techniques :
> 1. **Toujours** utiliser Early Stopping
> 2. **Toujours** essayer un peu de Weight Decay (0.0001 a 0.01)
> 3. Ajouter du Dropout si necessaire (0.1 a 0.5)
> 4. Utiliser la Data Augmentation si possible (surtout en vision)
> 5. Si rien ne marche : **collecter plus de donnees** !

### Conseil de pro : Le biais-variance tradeoff

> - **Biais eleve** (underfitting) : le modele est trop simple -> augmenter la capacite
> - **Variance elevee** (overfitting) : le modele est trop complexe -> regulariser
> - L'objectif est de trouver le **juste milieu** entre les deux